In [1]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data  # 这里 x 的维度是 [3, 1]
data.is_directed()

/home/seeyou/anaconda3/envs/pyg/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755853042/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [2]:
Data(x=[3, 1], edge_index=[2, 4])

Data(x=[2], edge_index=[2])

In [3]:
# Another Data
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
# If you want to write your indices this way, you should transpose and call contiguous
data = Data(x=x, edge_index=edge_index.t().contiguous())
data

Data(x=[3, 1], edge_index=[2, 4])

In [21]:
print(data.keys)
print(data['x'])
for key, item in data:
    print(f'{key} found in data')
print('edge_attr' in data)
print(len(data))
print(data.num_nodes)
print(data.num_node_features)
print(data.has_isolated_nodes())
print(data.has_self_loops())
print(data.is_directed())

['edge_index', 'x']
tensor([[-1.],
        [ 0.],
        [ 1.]])
x found in data
edge_index found in data
False
2
3
1
False
False
False


In [22]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

# Mini-batches

In [24]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

# 如果置 True，数据集会包含额外的连续边 attributes（如果存在）
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print(batch)
    print(batch.num_graphs)
    break

DataBatch(edge_index=[2, 4202], x=[1061, 21], y=[32], batch=[1061], ptr=[33])
32


In [28]:
from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
    # Batch is a column vector which maps each node to its respective graph in the batch
    # 批掩码，用于保持维护多节点并行计算，指示多少个节点属于同一张图
    print(data.batch.shape)
    x = scatter_mean(data.x, data.batch, dim=0)
    print(x.size())
    break

torch.Size([1200])
torch.Size([32, 21])


# Data Transforms 略

# Learning Methods on Graphs

In [31]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

# 第一个模型
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 实例化层，限定层的参数
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # 前向运算，输入的是数据
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [33]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7980
